In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/VRYNT/code/Word_Clustering'

Mounted at /content/gdrive
/content/gdrive/MyDrive/VRYNT/code/Word_Clustering


In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
! pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-o5vb7nnk
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-o5vb7nnk
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=a4b0d2c2cbb76dfdc5adb0cf32f0e323d0a6d68edff1d65c7cadbe65c341c746
  Stored in directory: /tmp/pip-ephem-wheel-cache-5gpjiojl/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
from pkg_resources import packaging
import re

print("Torch version:", torch.__version__)


Torch version: 1.12.1+cu113


In [ ]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Zero-Shot Image Classification

You can classify images using the cosine similarity (times 100) as the logits to the softmax operation.

In [ ]:
from torchvision.datasets import CIFAR100

dataLabels = np.array(CIFAR100(os.path.expanduser("~/.cache"),download=True).classes)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting /root/.cache/cifar-100-python.tar.gz to /root/.cache


In [ ]:
dataLabels = np.array([])
with open('imagenet1000_clsidx_to_labels.txt') as f:
  for line in f:
    word = re.split(''',|'|"|\n''',line)[1]
    dataLabels = np.append(dataLabels,word)    
print(dataLabels[:10])

['tench' 'goldfish' 'great white shark' 'tiger shark' 'hammerhead'
 'electric ray' 'stingray' 'cock' 'hen' 'ostrich']


In [ ]:
from gensim.models import KeyedVectors
word2vec_vectors = KeyedVectors.load("word_embedings/word2vec.wordvectors", mmap='r')
glove_vectors = KeyedVectors.load("word_embedings/glove50.wordvectors", mmap='r')

In [ ]:
data_tokensG = glove_vectors[dataLabels[0]]
data_labelsG = [dataLabels[0]]
data_idxNG = []

for i, word in enumerate(dataLabels[1:]):
  try:
    x = glove_vectors[word][np.newaxis,:]
    data_tokensG = np.vstack((data_tokensG,x))
    data_labelsG.append(word)
  except KeyError:
    wordsplit = re.split('-| ',word)
    try:
      data_tokensG = np.vstack((data_tokensG,glove_vectors["".join(wordsplit)][np.newaxis,:]))
      data_labelsG.append(word)
    except KeyError:
      try:
        data_tokensG = np.vstack((data_tokensG,np.mean(glove_vectors[wordsplit],axis=0)[np.newaxis,:]))
        data_labelsG.append(word)
      except KeyError:
        data_idxNG.append(i+1)
data_labelsG.append('miscellaneous')

data_tokensW = word2vec_vectors[dataLabels[0]]
data_labelsW = [dataLabels[0]]
data_idxNW = []

for i, word in enumerate(dataLabels[1:]):
  try:
    x = word2vec_vectors[word][np.newaxis,:]
    data_tokensW = np.vstack((data_tokensW,x))
    data_labelsW.append(word)
  except KeyError:
    wordsplit = re.split('-| ',word)
    try:
      data_tokensW = np.vstack((data_tokensW,word2vec_vectors["".join(wordsplit)][np.newaxis,:]))
      data_labelsW.append(word)
    except KeyError:
      try:
        data_tokensW = np.vstack((data_tokensW,np.mean(word2vec_vectors[wordsplit],axis=0)[np.newaxis,:]))
        data_labelsW.append(word)
      except KeyError:
        data_idxNW.append(i+1)
data_labelsW.append('miscellaneous')

In [ ]:
print('Not vectorized Glove:',len(data_idxNG))
print('Not vectorized word2vec:',len(data_idxNW))
# print(dataLabels[data_idxNW])
# print(data_tokensG.shape)

Not vectorized Glove: 134
Not vectorized word2vec: 38


In [ ]:
import csv
words = np.array([])
with open('gen_string_filled.csv', 'r') as file:
  csvreader = csv.reader(file)
  words = np.append(words,next(csvreader)[0].lower())
  for word in csvreader:
      words = np.append(words,word[0].lower())
print(len(words))

word_tokensG = glove_vectors[words[0]][np.newaxis,:]
word_idxNG = []
for i, word in enumerate(words[1:]):
  try:
    x = glove_vectors[word][np.newaxis,:]
    word_tokensG = np.vstack((word_tokensG,x))
  except KeyError:
    wordsplit = re.split('-| ',word)
    try:
      word_tokensG = np.vstack((word_tokensG,glove_vectors["".join(wordsplit)][np.newaxis,:]))
    except KeyError:
      try:
        word_tokensG = np.vstack((word_tokensG,np.mean(glove_vectors[wordsplit],axis=0)[np.newaxis,:]))
      except KeyError:
        word_tokensG = np.vstack((word_tokensG,np.zeros((1,50))))
        word_idxNG.append(i+1)

word_tokensW = word2vec_vectors[words[0]][np.newaxis,:]
word_idxNW = []
for i, word in enumerate(words[1:]):
  try:
    x = word2vec_vectors[word][np.newaxis,:]
    word_tokensW = np.vstack((word_tokensW,x))
  except KeyError:
    wordsplit = re.split('-| ',word)
    try:
      word_tokensW = np.vstack((word_tokensW,word2vec_vectors["".join(wordsplit)][np.newaxis,:]))
    except KeyError:
      try:
        word_tokensW = np.vstack((word_tokensW,np.mean(word2vec_vectors[wordsplit],axis=0)[np.newaxis,:]))
      except KeyError:
        word_tokensW = np.vstack((word_tokensW,np.zeros((1,300))))
        word_idxNW.append(i+1)

print(word_tokensG.shape)
print(word_tokensW.shape)

6894
(6894, 50)
(6894, 300)


In [ ]:
print('Not vectorized Glove:',len(word_idxNG))
print('Not vectorized word2vec:',len(word_idxNW))

Not vectorized Glove: 89
Not vectorized word2vec: 107


In [ ]:
print(words[word_idxNG])
print(words[word_idxNW])

['aggradation' 'aglet' 'alpenglow' 'alpenhorn' 'antling' 'backburn'
 'bloodflow' 'bootee' 'boxspring' 'brocolli' 'bunghole' 'chainstay'
 'checkroom' 'chiffonier' 'chromolithograph' 'colorlessness' 'comestible'
 'cosset' 'councilperson' 'covariate' 'disembodiment' 'dragonfruit'
 'duplexer' 'ecumenist' 'eicosanoid' 'elver' 'epauliere' 'ephyra'
 'fedelini' 'ficlet' 'floozie' 'formicarium' 'gamebird' 'hearthside'
 'homogenate' 'hydrocarb' 'initialise' 'jalapeã±o' 'jalapeã±o' 'lambkin'
 'leveret' 'lightscreen' 'ligula' 'lunchmeat' 'lyocell' 'macrofauna'
 'manacle' 'marionberry' 'mukluk' 'nestmate' 'nucleotidase' 'overnighter'
 'panpipe' 'pantologist' 'pantology' 'pattypan' 'pinworm' 'placode'
 'plier' 'polliwog' 'prostanoid' 'puggle' 'randomisation'
 'rehospitalisation' 'rehospitalization' 'reinscription' 'retrospectivity'
 'revascularisation' 'shoat' 'skean' 'sparerib' 'spiderling' 'sprag'
 'spume' 'stopsign' 'stot' 'strawman' 'subexpression' 'sunbonnet'
 'sunlamp' 'swanling' 'tailbud' "t

In [ ]:
#cosine similarity
similarity = word_tokensG @ data_tokensG.T
gCluster_idx = np.argmax(similarity,axis=1)

similarity = word_tokensW @ data_tokensW.T
wCluster_idx = np.argmax(similarity,axis=1)

for idx in word_idxNG:
  gCluster_idx[idx] = -1
for idx in word_idxNW:
  wCluster_idx[idx] = -1

In [ ]:
import pandas as pd
data_labelsG = np.array(data_labelsG)
df = pd.DataFrame({'Word': words,
                   'ClusterID': gCluster_idx,
                   'Cluster_Label': data_labelsG[gCluster_idx]})
df.to_csv('./clusters/Glove_cosine_sim_Cluster_ImageNet.csv',index=False)

data_labelsW = np.array(data_labelsW)
df = pd.DataFrame({'Word': words,
                   'ClusterID': wCluster_idx,
                   'Cluster_Label': data_labelsW[wCluster_idx]})
df.to_csv('./clusters/word2vec_cosine_sim_Cluster_ImageNet.csv',index=False)